# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [1]:
!pip install -qU datasets pinecone-client sentence-transformers torch

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [2]:
import torch
# set device to GPU if available
if torch.backends.mps.is_available():
    device = 'mps'  # Use Metal Performance Shaders (MPS) for Apple Silicon
elif torch.cuda.is_available():
    device = 'cuda'  # Use CUDA for NVIDIA GPUs (not applicable on Mac)
else:
    device = 'cpu' 

In [ ]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

/Users/weemoe/Documents/Ironhack/Week 7/Labs/day_2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from datasets import load_dataset

# Load the dataset in streaming mode
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

# Limit the number of examples processed (optional)
max_examples = 10  # Process only 100 examples

# Iterate over the streamed dataset
for idx, example in enumerate(wiki_data):
    # Do something with each example
    print(example)

    # Stop after reaching the max number of examples
    if idx >= max_examples - 1:
        break

{'wiki_id': 'Q7649565', 'start_paragraph': 20, 'start_character': 272, 'end_paragraph': 24, 'end_character': 380, 'article_title': 'Sustainable Agriculture Research and Education', 'section_title': "2000s & Evaluation of the program's effectiveness", 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}
{'wiki_id': 'Q1739333', 'start_paragraph': 18, 'start_character': 763, 

In [11]:
wiki_data

IterableDataset({
    features: ['wiki_id', 'start_paragraph', 'start_character', 'end_paragraph', 'end_character', 'article_title', 'section_title', 'passage_text'],
    n_shards: 1
})

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [12]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [14]:
history = wiki_data.filter(lambda x: x['section_title'] == 'History')

In [15]:
# filter only documents with History as section_title - Replace None with your code


In [20]:

next(iter(history))

{'wiki_id': 'Q2644349',
 'start_paragraph': 10,
 'start_character': 397,
 'end_paragraph': 10,
 'end_character': 534,
 'article_title': 'Taupo District',
 'section_title': 'History',
 'passage_text': 'was not until the 1950s that the region started to develop, with forestry and the construction of the Wairakei geothermal power station.'}

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [21]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # Extract the fields we need - article, section, and passage
    # Make sure d is a dictionary-like object and contains the necessary keys
    x = {
        'article_title': d.get('article_title'), 
        'section_title': d.get('section_title'), 
        'passage_text': d.get('passage_text')
    }
    docs.append(x)  # Add the extracted fields to the list `docs`
    
    # Increase the counter
    counter += 1
    
    # Break the loop after processing `total_doc_count` documents
    if counter >= total_doc_count:
        break

100%|█████████▉| 49999/50000 [03:09<00:00, 263.48it/s]


In [40]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head(500)

,article_title,section_title,passage_text
0,Taupo District,History,was not until the 1950s that the region starte...
1,The Bishop Wand Church of England School,History,The Bishop Wand Church of England School Histo...
2,Surface Hill Uniting Church,History,in perpetual reminder that work and worship go...
3,The Electras (band),History,"as its B-side. However, copies of the single, ..."
4,Swanton House,History,it. Lane provided funds for restoration by the...
...,...,...,...
495,"Bellerive, Tasmania",History,"from local sandstone, much of the original str..."
496,Caffe Mediterraneum,History,"the Med closed for renovations. As of 2016, it..."
497,Boticca,History,"raised $2.5M, led by ISAI and joined by Japane..."
498,Bohuslän,History,1658. The fortress of Carlsten was built in Ma...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [55]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'


In [56]:
# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [24]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [60]:
pc.delete_index(index_name)

In [61]:
index_name = "abstractive-question-answering"

In [62]:
import time

# check if the index_name exists
if index_name not in pc.list_indexes():
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=768,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )

# connect to index we created
index = pc.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [63]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
if torch.backends.mps.is_available():
    device = 'mps'  # Use Metal Performance Shaders (MPS) for Apple Silicon
elif torch.cuda.is_available():
    device = 'cuda'  # Use CUDA for NVIDIA GPUs (not applicable on Mac)
else:
    device = 'cpu'   # Use CPU
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [64]:
df = df.head(1000)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [67]:
# Function to remove or replace non-ASCII characters
import unicodedata

def normalize_string(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if ord(c) < 128)

# we will use batches of 64
batch_size = 64

#You will create embedding for the passage_text variable and be use to include the meta data in each batch
for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    end_idx = min(i + batch_size, len(df))
    # extract batch
    batch = df.iloc[i:end_idx]
    # generate embeddings for batch
    texts = batch['passage_text'].tolist()
    emb = retriever.encode(texts)
# check that we have all vectors in index
    meta = [{'article_title': row['article_title'], 'section_title': row['section_title'], 'passage_text': row['passage_text']} for _, row in batch.iterrows()]
    # create unique IDs
    ids = [normalize_string(str(row['article_title'])) for _, row in batch.iterrows()]

    # add all to upsert list
    to_upsert = [(ids[j], emb[j], meta[j]) for j in range(len(emb))]
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

index.describe_index_stats()

100%|██████████| 16/16 [00:12<00:00,  1.27it/s]


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 394}},
 'total_vector_count': 394}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [68]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [100]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(  # Perform the query on the Pinecone index
        vector=xq[0],  # The question's embedding
        top_k=top_k,   # How many top results to return
        include_metadata=True  # Include the metadata (e.g., context)
    )
    context = [match['metadata']['passage_text'] for match in xc['matches']]
    return context

In [103]:
def format_query(query, context):
    # Extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m}" for m in context]  # `m` here is already the passage_text
    # Concatenate all context passages
    context_str = " ".join(context)  # Joins passages into one string with a space

    # Step 3: Concatenate the query and context passages
    formatted_query = f"{query}\n\nContext:\n{context_str}"

    return formatted_query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [104]:
query = "what is 1+1?"
result = query_pinecone(query, top_k=1)
result

['a settlement, but as a leisure center.']

In [105]:
from pprint import pprint
result

['a settlement, but as a leisure center.']

In [106]:
result

['a settlement, but as a leisure center.']

In [107]:
# format the query in the form generator expects the input
query = format_query(query, result)
pprint(query)

'what is 1+1?\n\nContext:\n<P> a settlement, but as a leisure center.'


The output looks great. Now let's write a function to generate answers.

In [109]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [110]:
generate_answer(query)

'1+1 = 1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+'


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [112]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context)
generate_answer(query)

('The first wireless message was sent by a radio transmitter. The first radio '
 'transmitter was built in the late 1800s. The first wireless message was sent '
 'by a radio transmitter in the early 1900s')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [114]:
for doc in context:
    print(doc, end='\n---\n')

WJTI History The 1460 kHz frequency signed on the air in 1950 with the WRAC call sign. The owner of the station purchased another Racine station, WRJN-FM in 1969, changing it to WRAC-FM. A year later, the FM station flipped to a rock-leaning top 40 format as WRKR, and WRAC later adopted that call sign, simulcasting their FM sister station.
It was also for a brief time WWEG ("The Country Egg") before returning to WRKR and again simulcasting the FM signal. Later, there was a short lived Spanish format.
The station switched calls to WBZN on October 14, 1987, simulcasting its sister station's
---
network.  Other vendors such as Cisco created equipment that could be placed in routers to transport voice calls across a company network from site to site.  The termination of PBX circuits to be transported across a network and delivered to another phone system is traditionally referred to as Voice over IP (Voice over Internet Protocol or VoIP).  This design required special hardware on both ends

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [115]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

('COVID-19 is a synthetic opioid. It is a class of opiate that is used to '
 'treat opiate overdoses. It is a class of opiate that is used to treat op')


In [118]:
for doc in context:
    print(doc, end='\n---\n')

Α-Methylfentanyl History α-Methylfentanyl was initially discovered by a team at Janssen Pharmaceutica in the 1960s. In 1976, it began to appear mixed with "china white" heroin as an additive. It was first identified in the bodies of two drug overdose victims in Orange County, California, in December 1979, who appeared to have died from opiate overdose but tested negative for any known drugs of this type. Over the next year, there were 13 more deaths, and eventually the responsible agent was identified as α-methylfentanyl.
α-Methylfentanyl was placed on the Schedule I list in September 1981, only two years after its appearance
---
hockey team. The tenth WickFest will be held in early 2019 in Surrey, British Columbia.
---
release of version 1.79.
From version 1.76 the installer includes the adware OpenCandy.
In November 2017 version 1.96 was released. It is not clear who is the current code maintainer, and no contact details are provided. Four further versions were released by March 2018

Let’s finish with a final few questions.

In [ ]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The War of Currents was a series of events in the early 1900s between Edison '
 'and Westinghouse. The two companies were competing for the market share of '
 'electric power in the United States')


In [ ]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Neil Armstrong in 1969. He walked '
 'on the moon in 1969. He was the first person to walk on the moon.')


In [ ]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $10 billion to build.')


As we can see, the model can generate some decent answers.

#### Add a few more questions

In [119]:
query = "what is Methylfentanyl?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

('Methylfentanyl is a synthetic opiate. It is a very potent opiate, and it is '
 'used to treat opiate addiction. It is also used to treat pain. It is')


In [121]:
query = "who is Messi ?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

("He's a very good player. He's a very good player. He's a very good player. "
 "He's a very good player. He's a very good player. He's a")
